In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

a = cobra.io.read_sbml_model("iEC1344_C.xml")

Academic license - for non-commercial use only - expires 2023-02-21
Using license file /home/schwartzlab/gurobi.lic


In [5]:
csvFile = pd.read_csv('biomass_per_flux_for_reaction_ALAALAD.csv') #this is flat and then decreasing i.e. saturating
i=0
l= list()

while i < 201:
    inf= csvFile.iloc[i,  2]
    flt= float(csvFile.iloc[i,  2])
    
    if csvFile.iloc[i,  2] == flt:
        l.append(csvFile.iloc[i,  2])
    i= i+1

csvFile2= pd.DataFrame (l, columns = ['Biomass'])
csvFile2

my_series = csvFile2['Biomass'].squeeze()
type(my_series)

my_series

if my_series.is_monotonic_decreasing == False and my_series.is_monotonic == False:
    print("This reaction is both increasting and decreasing.")

This reaction is both increasting and decreasing.


In [2]:
int_reactions= list()
i= -1
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()

for r in a.reactions:
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    
    if v == 1:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)

print(one)
print(zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D

['EX_cm_e', 'EX_cmp_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'EX_csn_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'DM_oxam_c', 'EX_glyb_e', 'EX_gthox_e', 'EX_gtp_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_12ppd__R_e', 'EX_14glucan_e', 'EX_15dap_e', 'EX_23ccmp_e', 'EX_23cump_e', 'EX_26dap__M_e', 'EX_h2o2_e', 'EX_h2s_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_3cmp_e', 'EX_3hpp_e', 'EX_3ump_e', 'EX_hg2_e', 'EX_his__L_e', 'EX_hom__L_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_LalaDgluMdap_e', 'EX_cys__L_e', 'EX_LalaDgluMdapDala_e', 'EX_cytd_e', 'EX_ile__L_e', 'EX_indole_e', 'EX_acgal_e', 'EX_inost_e', 'EX_isetac_e', 'EX_acgal1p_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_leu__L_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_dcmp_e', 'EX_agm_e', 'EX_lipoate_e', 'EX_dcyt_e', 'EX_lys__L_e', 'EX_ala_B_e', 'EX_malt_e', 'EX_m

,1 feasible value
0,EX_cm_e
1,EX_cmp_e
2,EX_cobalt2_e
3,EX_colipa_e
4,EX_colipap_e
...,...
1569,AMID3
1570,SUCP
1571,LTARTDH
1572,UAG4Ei


In [3]:
i= 0
diff_saturated= list()
diff_indecr= list()
flat= list()
r = a.reactions[0]


for r in a.reactions:
    
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])


    if len(d)> 1:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.02 and d.iloc[n2,  0] < max(d["Biomass"])+0.02:
                count= count+1
    
            n2= n2+1
        
        if count> 0.8*len(d) and count< 0.95*len(d):
            diff_saturated.append(r_id)

        elif count> 0.95*len(d):
            flat.append(r_id)
        
        else:
            diff_indecr.append(r_id)

    
    i= i+1

flat
diff_indecr
diff_saturated

['EX_h2_e',
 'EX_ppt_e',
 'ACKr',
 'ACOLIPAabctex',
 'ALAALAD',
 'ALATA_D2',
 'ALATA_L2',
 'ALDD2x',
 'ATPM',
 'CHLabcpp',
 'CLIPAabctex',
 'CAT',
 'CBPS',
 'CD2abcpp',
 'COBALT2abcpp',
 'COLIPAabctex',
 'CRNDt2rpp',
 'CU1abcpp',
 'CYSabc2pp',
 'ECA4COLIPAabctex',
 'ENLIPAabctex',
 'ARGabcpp',
 'ASNN',
 'FE2abcpp',
 'G3PD5',
 'G3PD6',
 'G3PD7',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLCabcpp',
 'ASPT',
 'GALabcpp',
 'GLYBabcpp',
 'GLYC3Pabcpp',
 'IDOND2',
 'HG2abcpp',
 'HPYRRy',
 'Kabcpp',
 'LDH_D2',
 'LIPAabctex',
 'LYSabcpp',
 'MDH2',
 'MDH3',
 'MG2uabcpp',
 'NADH10',
 'NADH16pp',
 'NADH5',
 'NADH9',
 'NADPHQR2',
 'NADPHQR3',
 'NADPHQR4',
 'NADTRHD',
 'NI2abcpp',
 'NI2uabcpp',
 'NTP1',
 'NTP10',
 'NTP3',
 'NTP5',
 'NTPP1',
 'NTPP2',
 'NTPP3',
 'NTPP4',
 'NTPP5',
 'NTPP6',
 'NTPP7',
 'NTPP8',
 'NTRIR3pp',
 'NTRIR4pp',
 'ORNabcpp',
 'PDH',
 'PTAr',
 'PTRCabcpp',
 'SPODM',
 'SULabcpp',
 'THRA2',
 'THRabcpp',
 'XYLabcpp',
 'ZN2abcpp',
 'ZNabcpp',
 'POX',
 'PPA',
 'PPA2',
 'PPPGO',
 'DURAD',
 'P

In [4]:
i= 0
r_id= diff_saturated[i]
diff_saturatedFINAL= list()
Ushaped_reactions= list()

for r_id in diff_saturated:

    l= list()
    n=0
    n2=0
    r_id = diff_saturated[i]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(r_id))

    while n < 201:
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    
    if d.iloc[0,0] > d.iloc[1,0]-0.02 and d.iloc[0,0] < d.iloc[1,0]+0.02 or\
       d.iloc[1,0] > d.iloc[0,0]-0.02 and d.iloc[1,0] < d.iloc[0,0]+0.02 or\
       d.iloc[len(d)-2,0] > d.iloc[len(d)-1,0]-0.02 and d.iloc[len(d)-2,0] < d.iloc[len(d)-1,0]+0.02 or\
       d.iloc[len(d)-1,0] > d.iloc[len(d)-2,0]-0.02 and d.iloc[len(d)-1,0] < d.iloc[len(d)-2, 0]+0.02:
        diff_saturatedFINAL.append(r_id)
    
    else:
        Ushaped_reactions.append(r_id)
    
    i= i+1

diff_saturatedFINAL
Ushaped_reactions

['EX_h2_e',
 'EX_ppt_e',
 'ACKr',
 'ALATA_L2',
 'CAT',
 'CU1abcpp',
 'ASNN',
 'ASPO4',
 'Kabcpp',
 'NADH16pp',
 'NI2uabcpp',
 'NTRIR3pp',
 'NTRIR4pp',
 'PDH',
 'PTAr',
 'SPODM',
 'SULabcpp',
 'ZNabcpp',
 'PPA',
 'PYRDC']

In [5]:
i2= 0
r_id = diff_indecr[0]
mon_incr= list()
mon_decr= list()
one_feasible_value= list()

for r_id in diff_indecr:                    #this code finds reactions that are either stable or constantly decrease
    
    i=0
    r_id = diff_indecr[i2]
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(r_id))
    l= list()

    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    my_series = csvFile2['Biomass'].squeeze()
    type(my_series)

    if len(csvFile2['Biomass'])>1:
        
        if my_series.is_monotonic_decreasing ==True:
            mon_decr.append(r_id)
        elif my_series.is_monotonic ==True:
            mon_incr.append(r_id)

    
    i2= i2 + 1

mon_decr


['EX_glcur1p_e',
 'EX_gln__L_e',
 'EX_glu__L_e',
 'EX_glyald_e',
 'EX_glyc3p_e',
 'EX_gmp_e',
 'EX_gsn_e',
 'EX_gthrd_e',
 'EX_gua_e',
 'EX_3amp_e',
 'EX_3gmp_e',
 'EX_hxan_e',
 'EX_LalaDglu_e',
 'EX_LalaLglu_e',
 'EX_imp_e',
 'EX_acald_e',
 'EX_ins_e',
 'EX_acser_e',
 'EX_adn_e',
 'EX_akg_e',
 'EX_dgmp_e',
 'EX_amp_e',
 'EX_dgsn_e',
 'EX_dimp_e',
 'EX_cit_e',
 'EX_xan_e',
 '2AGPA120tipp',
 '2AGPA140tipp',
 '2AGPA141tipp',
 '2AGPA160tipp',
 '2AGPA161tipp',
 'EX_etha_e',
 '2AGPA180tipp',
 '2AGPA181tipp',
 '2AGPE120tipp',
 '2AGPE140tipp',
 '2AGPE141tipp',
 '2AGPE160tipp',
 '2AGPE161tipp',
 '2AGPE180tipp',
 '2AGPE181tipp',
 '2AGPG120tipp',
 '2AGPG140tipp',
 '2AGPG141tipp',
 '2AGPG160tipp',
 '2AGPG161tipp',
 '2AGPG180tipp',
 '2AGPG181tipp',
 'EX_ura_e',
 'EX_urea_e',
 '2MAHMP',
 'EX_fruur_e',
 'AACPS5',
 'AACPS6',
 'AACPS7',
 'AACPS8',
 'AACPS9',
 'ABTA',
 'EX_g1p_e',
 'AACPS1',
 'AACPS2',
 'AACPS3',
 'AACPS4',
 'ACPPAT120',
 'ACPPAT140',
 'ACPPAT141',
 'AGPAT181',
 'AGPR',
 'ACPPAT180',
 

In [6]:
mon_incr

['EX_cpgn_e',
 'EX_glcr_e',
 'EX_23dappa_e',
 'EX_cynt_e',
 'EX_mal__D_e',
 'EX_no_e',
 'EX_no2_e',
 'EX_dmso_e',
 'EX_no3_e',
 'EX_o2_e',
 'EX_pser__L_e',
 'EX_ser__D_e',
 'EX_tartr__D_e',
 'EX_tartr__L_e',
 'EX_fald_e',
 'EX_fe3dcit_e',
 '2DGULRGx',
 'EX_fum_e',
 'ACGK',
 'ACGS',
 'ACOATA',
 'EX_galct__D_e',
 'AICART',
 'AIRC2',
 'ADK4',
 'ADSL1r',
 'ADSL2r',
 'ADSS',
 'ALPATE160pp',
 'ALPATG160pp',
 'AGPAT160',
 'AGPAT161',
 'APG3PAT160',
 'ATHRDHr',
 'APG3PAT161',
 'CHLt2pp',
 'BETALDHy',
 'BSORx',
 'CITtex',
 'CLPNH120pp',
 'CLPNH140pp',
 'CLPNH141pp',
 'CLPNH160pp',
 'CLPNH161pp',
 'CLPNH180pp',
 'CLPNH181pp',
 'CBMKr',
 'CPGNR1',
 'ARBTNR1',
 'CSNt2pp',
 'CTBTt2rpp',
 'CTPS2',
 'CUt3',
 'DCTPD',
 'CYSS',
 'DHAtex',
 'DHAtpp',
 'DHORD5',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DOGULNR',
 'ARGDC',
 'EAR60x',
 'EAR80x',
 'DSBAO2',
 'DSERDHr',
 'ENO',
 'EAR100x',
 'EAR120x',
 'ENTERES',
 'ENTERES2',
 'EAR121x',
 'EAR140x',
 'EAR141x',
 'EAR160x',
 'EAR161x',
 'EAR180x',
 '

In [7]:
i2=0
mon_decr_and_stable= list()
mon_decr_only= list()

while i2< len(mon_decr):  #this code separates reactions that may have flat points from reactions that always decrease
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(mon_decr[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_decr_and_stable.append(mon_decr[i2])
    
    if boolean == False:
        mon_decr_only.append(mon_decr[i2])
    
    i2= i2+1

mon_decr_and_stable


['EX_glcur1p_e',
 'EX_glyald_e',
 'EX_gua_e',
 'EX_3amp_e',
 'EX_amp_e',
 'EX_dimp_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'ALAabcpp',
 'ALDD2y',
 'ASPabcpp',
 'ASPt2_2pp',
 'ASPt2_3pp',
 'CMtpp',
 'CRNDabcpp',
 'CRNabcpp',
 'CTBTabcpp',
 'CYTBD2pp',
 'CYTBDpp',
 'ARBabcpp',
 'ARBt3ipp',
 'DOXRBCNtpp',
 'ARGDCpp',
 'FBA3',
 'FE2t2pp',
 'ASNabcpp',
 'FRD3',
 'FUMt2_3pp',
 'FUSAtpp',
 'GLUabcpp',
 'GUAt2pp',
 'ILEabcpp',
 'HISabcpp',
 'LCTSt3ipp',
 'LEUabcpp',
 'MALt2_3pp',
 'MINCYCtpp',
 'MG2tpp',
 'NAt3_1p5pp',
 'NAt3_2pp',
 'NOVBCNtpp',
 'NO3R2bpp',
 'PROabcpp',
 'RFAMPtpp',
 'SUCCt2_3pp',
 'SUCDi',
 'THIORDXi',
 'TTRCYCtpp',
 'XANt2pp',
 'VALabcpp',
 'DARBabcpp']

In [8]:
i2=0
mon_incr_and_stable= list()
mon_incr_only= list()

while i2< len(mon_incr):  #this code separates reactions that may have flat points from reactions that always increase
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(mon_incr[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)

    boolean = csvFile2["Biomass"].duplicated().any()
    
    if boolean == True:
        mon_incr_and_stable.append(mon_incr[i2])
    
    if boolean == False:
        mon_incr_only.append(mon_incr[i2])
    
    i2= i2+1

mon_incr_and_stable


['DHORD5',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'DSBAO2',
 'FRUpts2pp',
 'HYD2pp',
 'HYD3pp',
 'NO3R1pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp']

In [9]:
d.iloc[len(d)-1,0]

0.0356413104968205

In [9]:
set1= set(diff_indecr)
set2= set(mon_decr)
set3= set(mon_incr)
inANDdecr= list(set1-set2-set3)
inANDdecr

['MACPD',
 'EX_lcts_e',
 'NTD11',
 'GLYCDx',
 'EX_acgam1p_e',
 'EX_f6p_e',
 'NI2tpp',
 'MN2tpp',
 'RNDR1b',
 'GAPD',
 'EX_thr__L_e',
 'EX_gal_bD_e',
 'EX_ppal_e',
 'UGLYCH',
 'EX_succ_e',
 'CYTD',
 'ALAALAr',
 'EX_pyr_e',
 'EX_arab__L_e',
 'RNDR4b',
 'EX_octa_e',
 'DURIK1',
 'EX_galctn__D_e',
 'EX_acmum_e',
 'F6PA',
 '2AGPGAT161',
 'PPS',
 'NTD2',
 'EX_galctn__L_e',
 'DASYN161',
 'EX_dha_e',
 'EX_ttdcea_e',
 'RNTR1c2',
 'EX_ser__L_e',
 'GF6PTA',
 'EX_hxa_e',
 'EX_acac_e',
 'NADH17pp',
 'H2Otex',
 'GLYK',
 'CYTK1',
 'EX_ocdcea_e',
 'APH181',
 'EX_acmana_e',
 'ATPS4rpp',
 'PFL',
 'MALt2_2pp',
 'OMMBLHX3',
 'EX_gam6p_e',
 'Htex',
 'NADH18pp',
 '2AGPGAT180',
 'APH120',
 'APH160',
 'RNDR1',
 'EX_glyc_e',
 'EX_glyc2p_e',
 'PUNP2',
 'CYTBO3_4pp',
 'PA160abcpp',
 'APH180',
 'DGK1',
 'PPK2',
 'RNTR2c',
 'THD2pp',
 'FBP',
 'Kt2pp',
 'GDPDPK',
 'EX_gam_e',
 'CMPN',
 'NMNN',
 'MALS',
 'FUM',
 'PIuabcpp',
 'PGK',
 'H2Otpp',
 'LYSt2pp',
 'FRD2',
 'LDH_D',
 'CU2tpp',
 'EX_galt_e',
 'UMPK',
 'EX_acnam

In [10]:
i2=0
l2= list()
inANDdecr_and_stable= list()
inANDdecr_only= list()

while i2< len(inANDdecr):  #this code separates reactions that may have flat points from reactions that always increase and decrease
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(inANDdecr[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
    
    while i < len(csvFile2)-1:
        if csvFile2.iloc[i,  0] == csvFile2.iloc[i+1,  0] and inANDdecr[i2] not in l2:
            l2.append(inANDdecr[i2])
            
        i=i+1
        
    i2= i2+1

l2

['EX_f6p_e',
 'EX_arab__L_e',
 'EX_octa_e',
 'EX_acmum_e',
 'EX_dha_e',
 'EX_hxa_e',
 'NADH17pp',
 'EX_gam6p_e',
 'NADH18pp',
 'EX_gam_e',
 'PIuabcpp',
 'FRD2',
 'EX_g6p_e',
 'GLYCTO4',
 'EX_ptrc_e',
 'FDH5pp',
 'EX_tre_e',
 'MNt2pp',
 'EX_idon__L_e',
 'EX_xyl__D_e',
 'EX_man6p_e',
 'EX_glcn_e',
 'GLYCTO3',
 'EX_lyx__L_e',
 'EX_glyc__R_e',
 'EX_xylu__L_e',
 'EX_4abut_e',
 'NO3R1bpp',
 'FBA',
 'CITL',
 'TALA']

In [11]:
set1= set(inANDdecr)
set2= set(l2)
l_left= list(set1-set2)
l_left

['MACPD',
 'NTD11',
 'GLYCDx',
 'RNDR1b',
 'GAPD',
 'CYTD',
 'EX_ppal_e',
 'ALAALAr',
 'EX_succ_e',
 'EX_pyr_e',
 'EX_gal_bD_e',
 'RNDR4b',
 'F6PA',
 'PPS',
 'EX_ser__L_e',
 'GF6PTA',
 'EX_acac_e',
 'H2Otex',
 'GLYK',
 'EX_ocdcea_e',
 'APH181',
 'ATPS4rpp',
 'PFL',
 'MALt2_2pp',
 '2AGPGAT180',
 'APH160',
 'EX_glyc_e',
 'CYTBO3_4pp',
 'PA160abcpp',
 'APH180',
 'DGK1',
 'PPK2',
 'Kt2pp',
 'NMNN',
 'CMPN',
 'FUM',
 'MALS',
 'PGK',
 'EX_galt_e',
 'UMPK',
 'EX_acnam_e',
 'NADK',
 'HCO3E',
 'RNDR4',
 'GLUDC',
 'RNDR2b',
 'ALAR',
 'EX_12ppd__S_e',
 'EX_rmn_e',
 'EX_ala__D_e',
 'EX_ala__L_e',
 'G3PD2',
 'NNATr',
 'SSALy',
 'GLYAT',
 'SUCASPtpp',
 'GNK',
 'TMPK',
 'ACPPAT161',
 'ARBt2rpp',
 'RPE',
 'PYNP2r',
 'EX_ac_e',
 '2AGPEAT181',
 'EX_mnl_e',
 'EX_glcur_e',
 'OMPHHX3',
 'FE2tpp',
 'EX_dca_e',
 'PA181abcpp',
 'PPC',
 '2AGPEAT160',
 'NDPK8',
 'CYTK2',
 'EX_alltn_e',
 '2AGPEAT120',
 'GLUDy',
 'EX_g3pe_e',
 'EX_3hcinnm_e',
 'DTMPK',
 'NI2t3pp',
 'CS',
 'EX_glyclt_e',
 'GND',
 'RNDR2',
 'CD2t3p

In [12]:
i2=0
l3= list()
inANDdecr_and_stable= list()
inANDdecr_only= list()

while i2< len(l_left):  #this code separates reactions that may have flat points from reactions that always increase and decrease
    
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_study_conditions.csv'.format(l_left[i2]))
    i=0
    l= list()
    
    while i < 201:
        inf= csvFile.iloc[i,  2]
        flt= float(csvFile.iloc[i,  2])
    
        if csvFile.iloc[i,  2] == flt:
            l.append(csvFile.iloc[i,  2])
        
        i= i+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    i=0
        
    while i < len(csvFile2)-1:
        
        if csvFile2.iloc[i,  0] != csvFile2.iloc[i+1,  0] and l_left[i2] not in l3:
            l3.append(l_left[i2])
        
        i=i+1
        
    i2= i2+1

l3

['MACPD',
 'NTD11',
 'GLYCDx',
 'RNDR1b',
 'GAPD',
 'CYTD',
 'EX_ppal_e',
 'ALAALAr',
 'EX_succ_e',
 'EX_pyr_e',
 'EX_gal_bD_e',
 'RNDR4b',
 'F6PA',
 'PPS',
 'EX_ser__L_e',
 'GF6PTA',
 'EX_acac_e',
 'H2Otex',
 'GLYK',
 'EX_ocdcea_e',
 'APH181',
 'ATPS4rpp',
 'PFL',
 'MALt2_2pp',
 '2AGPGAT180',
 'APH160',
 'EX_glyc_e',
 'CYTBO3_4pp',
 'PA160abcpp',
 'APH180',
 'DGK1',
 'PPK2',
 'Kt2pp',
 'NMNN',
 'CMPN',
 'FUM',
 'MALS',
 'PGK',
 'EX_galt_e',
 'UMPK',
 'EX_acnam_e',
 'NADK',
 'HCO3E',
 'RNDR4',
 'GLUDC',
 'RNDR2b',
 'ALAR',
 'EX_12ppd__S_e',
 'EX_rmn_e',
 'EX_ala__D_e',
 'EX_ala__L_e',
 'G3PD2',
 'NNATr',
 'SSALy',
 'GLYAT',
 'SUCASPtpp',
 'GNK',
 'TMPK',
 'ACPPAT161',
 'ARBt2rpp',
 'RPE',
 'PYNP2r',
 'EX_ac_e',
 '2AGPEAT181',
 'EX_mnl_e',
 'EX_glcur_e',
 'OMPHHX3',
 'FE2tpp',
 'EX_dca_e',
 'PA181abcpp',
 'PPC',
 '2AGPEAT160',
 'NDPK8',
 'CYTK2',
 'EX_alltn_e',
 '2AGPEAT120',
 'GLUDy',
 'EX_g3pe_e',
 'EX_3hcinnm_e',
 'DTMPK',
 'NI2t3pp',
 'CS',
 'EX_glyclt_e',
 'GND',
 'RNDR2',
 'CD2t3p

In [103]:
d1= pd.DataFrame (mon_decr, columns = ['Reaction names'])
d2= pd.DataFrame (flat, columns = ['Reaction names'])
d3= pd.DataFrame (diff_saturatedFINAL, columns = ['Reaction names'])
d4= pd.DataFrame (Ushaped_reactions, columns = ['Reaction names'])
d5= pd.DataFrame (mon_decr_and_stable, columns = ['Reaction names'])
d6= pd.DataFrame (mon_decr_only, columns = ['Reaction names'])
d7= pd.DataFrame (mon_incr_and_stable, columns = ['Reaction names'])
d8= pd.DataFrame (mon_incr_only, columns = ['Reaction names'])
d9= pd.DataFrame (l2, columns = ['Reaction names'])
d10= pd.DataFrame (l3, columns = ['Reaction names'])

D.to_csv("one feasible value reactions study conditions.csv")
d1.to_csv("monotonically decreasing reactions study conditions.csv")
d2.to_csv("flat study conditions.csv")
d3.to_csv("diff_saturated study conditions.csv")
d4.to_csv("Ushaped_reactions study conditions.csv")
d5.to_csv("monotonically decreasing and stable reactions study conditions.csv")
d6.to_csv("monotonically decreasing only reactions study conditions.csv")
d7.to_csv("monotonically increasing and stable reactions study conditions.csv")
d8.to_csv("monotonically increasing only reactions study conditions.csv")
d9.to_csv("increasing and decreasing and stable reactions study conditions.csv")
d10.to_csv("increasing and decreasing only reactions study conditions.csv")


In [26]:
len(Ushaped_reactions)

20